## Generative Model for Synchrotron Emission from Relativistic Jets

Radio interferometers generate raw data in the form of correlated visilibility ((u,v) data), or Fourier transforms of the sky brightness distribution. An inverse Fourier transform can then be computed in order to obtain an image of the sky. 

The visibility function for a two element interferometer is given by 

\begin{equation}
V = \int I_{\nu} exp(-2\pi i \cdot \vec{b} \hat{s}/\lambda) d\Omega
\end{equation}


where $\vec{b}$ is the baseline length, $\hat{s}$ is the direction in which the antennas point, $I_{\nu}$ is the brightness distribution on the sky, $\lambda$ is the wavelength, and $d\Omega$ is the solid angle over which the source extends.


In beginning with the (u,v) data, there are a numer of error terms which arise. 

1. Additive errors due to interference ($\epsilon_{Int}$), cross-talk ($\epsilon_{CT}$), receiver noise ($\epsilon_{rx}$), and correlator offsets ($\epsilon_{fx}$). Interference in the form of fluctuating intensities and crosstalk, in which antennas in an array communicate with one another transmitting artificial data, lead to errors in the final image on large angular scales. In the case of receiver noise, deconvolution can lead to the redistribution of noise features across an image, confusing astrophysical sources with noise artifacts.


2. Multiplicative errors due to phase errors ($\epsilon_{\phi}$) from the atmosphere and ionosphere and errors in the calibration of amplitude or phase ($\epsilon_{GAIN}$).$ Varying indices of refraction across unique lines of sight to individual antennas can lead to phase delays which are not representative of the source structure. Errors which may arise in calibration of the amplitude and phase of each antenna will be convolved with the final image.


3. Bandwidth and Time-average smearing.  In practice, the frequency bandwidth of a visibility is singular, ie. corresponding to a single frequency. Since the passband of the antenna instead characterizes a range of frequencies, there is some resultant radial broadening of the image, ie. chromatic aberration ($\epsilon_{bs}$). Similarly, time-averaging of visibility functions leads to elongated structures in the image ($\epsilon_{ts}$).


4. Baseline errors. Baseline errors ($\epsilon_{BL}$)lead to baseline dependent offsets in the data. A distribution of varying baseline offsets will lead to an artifical sidelobe response that is baseline dependent.


5. Pointing errors. Errors in the pointing ($\epsilon_{pt}$)between antennas can lead to amplitude fluctuations that vary as a function of time.


6. Computational Errors.The Fast Fourier Transform (FFT) algorithm (used to obtain the final image) makes some approximations in determining the Fourier transform, and introduces additional effects ($\epsilon_{FFT}$) via implementation of aliasing and convolution within the algorithm. 


Our generative model for describing the synchrotron emission of a relativistic jet can therefore be written as



\begin{equation}
M_{gen}(F_\nu,\nu_a,\nu_m) = M(F_\nu,\nu_a,\nu_m) \prod_{\mathrm{antennas}} \epsilon_{bs} \epsilon_{pt} \epsilon_{FFT} \epsilon_{\phi} \epsilon_{ts} \epsilon_{GAIN} + \sum_{antennas} \epsilon_{Int} + \epsilon_{CT} + \epsilon_{rx} + \epsilon_{fx} + \epsilon_{BL}
\end{equation}


where M is the model which describes the synchrotron spectra as a function of the normalization flux factor $F_\nu$, the self absorption frequency $\nu_a$, and the characteristic elecron frequency $\nu_m$. Due to the time-variable nature of the location of the break frequencies, the shape of the spectra changes depending on the relative nature of $\nu_a$ and $\nu_m$. When $\nu_m$ is greater than $\nu_a$, the spectrum is given by


\begin{equation}
F_1 = F_\nu \Bigg[\Big(\frac{\nu}{\nu_a}\Big)^{-s_1\beta_1} + \Big(\frac{\nu}{\nu_a}\Big)^{-s_2\beta_2}\Bigg] \Bigg[1 + \Big(\frac{\nu}{\nu_m}\Big)^{s_2(\beta_2-\beta_3)}\Bigg]^{-1/s_2}
\end{equation}

where $s_1$ and $s_2$ are smoothing parameters which describe the spectrum across each break, and $\beta_1$, $\beta_2$, and $\beta_3$ are the power law indices for each region of the spectrum. As $\nu_a$ evolves and becomes greater than $\nu_m$, the shape of the spectrum can no longer be characterized by the above model, and is instead best described by 

 \begin{equation}
F_2 = F_\nu (\nu_m) \Bigg[ \Big(\frac{\nu}{\nu_m}\Big)^2 exp\Big[-s \Big(\frac{\nu}{\nu_m}\Big)^{2/3}\Big] +  \Big(\frac{\nu}{\nu_m}\Big)^{5/2}\Bigg] \times \Bigg[1 + \Big(\frac{\nu}{\nu_a}\Big)^{-s_2 (\beta_2 - \beta_3)}\Bigg]^{-1/s_2}
\end{equation}


When $\nu_a$ and $\nu_m$ are sufficiently far apart, the spectra are well constrained by the above equations. As the frequencies evolve ($\nu_a$ moves to higher frequencies as $\nu_m$ decreases), neither individual spectra is appropriate. We therefore adopt a weighting scheme given by 

\begin{equation}
F(F_\nu,\nu_a,\nu_m) = \frac{F_1\frac{\nu_m}{\nu_a}^2 + F_2\frac{\nu_a}{\nu_m}^2}{\frac{\nu_a}{\nu_m}^2 + \frac{\nu_m}{\nu_a}^2}.
\end{equation}

Depending on the relative nature of either break frequency, each individual spectra contributes to the total observed spectrum accordinly.


Given that the errors can be described as Guassian random processes, we arrive at our familiar result for the likelihood function.

\begin{equation}
\mathcal{L}(F_\nu,\nu_a,\nu_m) \propto \prod_{i=1}^N \exp(-\frac{(d_i-M_{gen}(F_\nu,\nu_a,\nu_m))^2}{2 \sigma_i^2})
\end{equation}

For the purposes of this repository, the errors which apply to the raw visibilities have been propagated through to final flux density measurements (in mJy) and their respective uncertainties. Therefore, modeling of the data will in practice entail comparing the flux density measurements directly to the synchrotron model, with an additional term which describes the fractional amount f by which the variances are underestimated. The complete log likelihood function is therefore given by 

\begin{equation}
ln p(y|x,\sigma,F_\nu,\nu_a,\nu_m, f) = -\frac{1}{2}\sum_{n} \frac{(y_n - F(F_\nu,\nu_a,\nu_m))^2 + ln 2\pi(\sigma_n^2 + f^2 F(F_\nu,\nu_a,\nu_m)^2 )}{2 \sigma_n^2 + f^2 F(F_\nu,\nu_a,\nu_m)^2 }
\end{equation}

where $F_\nu,\nu_a,\nu_m$ are the inferred values.